# crop 된 이미지를 유사한 특성이 가진 1:1 크기의 파일로 변경

- gray 스케일로 할지 안할지...



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import cv2
from google.colab.patches import cv2_imshow

from tqdm import tqdm

import os
import warnings
warnings.filterwarnings(action='ignore')

In [43]:
csv_path = '/content/drive/MyDrive/project3/data/traindata/read_file'

csv_files = os.listdir(csv_path)

# Filter out the CSV files
csv_files = [file for file in csv_files if file.endswith('.csv')]

# Initialize an empty list to hold the dataframes
temp = []

# Read each CSV file and append to the list
for csv_file in csv_files[50:51]:
    file_path = os.path.join(csv_path, csv_file)
    df = pd.read_csv(file_path)
    temp.append(df)



In [46]:
# Concatenate all dataframes
df = pd.concat(temp, ignore_index=True)
df = df[ (df['type'] == 'crop')]
df.reset_index(drop=True, inplace=True)
df['file_path'][0]

,file_path,type
0,/content/drive/MyDrive/project3/data/traindata...,crop
1,/content/drive/MyDrive/project3/data/traindata...,crop
2,/content/drive/MyDrive/project3/data/traindata...,crop
3,/content/drive/MyDrive/project3/data/traindata...,crop
4,/content/drive/MyDrive/project3/data/traindata...,crop
5,/content/drive/MyDrive/project3/data/traindata...,crop
6,/content/drive/MyDrive/project3/data/traindata...,crop
7,/content/drive/MyDrive/project3/data/traindata...,crop
8,/content/drive/MyDrive/project3/data/traindata...,crop
9,/content/drive/MyDrive/project3/data/traindata...,crop


## 그림을 정사각형으로 바꾸고자 한다.
1. 1:1형태의 patch를 만든다.
    - 지금은 width, height 중 짧은 축
    - (width, height 중 짧은 축을 기준으로 20pixel 작은)
2. patch를 x축 / y축으로 20씩 이동시켜서 histgram을 그리고
3. 원본의 histogram과 유사한 patch를 선택
4. 이를 새로운 이미지로 저장
    - 컬러 / gray 2가지 방식으로 patch를 선택하고,
    - 두 경우 다 저장


In [38]:
# histogram 그리기
def calculate_histogram(image):
    # Gray Scale 일떄
    if len(image.shape) == 2:
        histogram = cv2.calcHist([image], # 이미지
                                 [0], # 사용될 채널
                                 None, # 마스크
                                 [256], #bins
                                 [0, 256]) #각 채널의 최소값 / 최대값
    else:  # 컬러 이미지
        histogram = cv2.calcHist([image],
                                 [0, 1, 2],
                                 None,
                                 [256, 256, 256],
                                 [0, 256, 0, 256, 0, 256])

    # 히스토그램 정규화
    cv2.normalize(histogram, histogram)
    return histogram


# 원본과 가장 비슷한 hist를 갖는 그림을 선정
def find_best_match(image, target_hist, patch_size):

    # 가장 비슷한 사이즈를 정하기 위한 준비
    min_diff = float('inf')
    best_x, best_y = 0, 0

    # 이미지 크기
    # Gray Scale 일떄
    if len(image.shape) == 2:
        height, width = image.shape
    # 컬러 이미지
    else:
        height, width, _ = image.shape
    # patch의 위치를 20씩 이동시키며 비교
    for y in range(0, height - patch_size, 20):
        for x in range(0, width - patch_size, 20):
            patch = image[y:y + patch_size, x:x + patch_size]
            patch_hist = calculate_histogram(patch)
            diff = cv2.compareHist(target_hist, patch_hist, cv2.HISTCMP_BHATTACHARYYA)
            if diff < min_diff:
                min_diff = diff
                best_x, best_y = x, y

    return best_x, best_y

def extract_best_patch(image):

    image_size_adj = 20

    # gray로 판별하는 방법
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    height_g, width_g = image_gray.shape
    patch_size_g = min(height_g - image_size_adj, width_g - image_size_adj)  # 1:1 비율의 패치 크기 결정(원본보다 20 작게)
    target_hist_g = calculate_histogram(image_gray)

    best_x_g, best_y_g = find_best_match(image_gray, target_hist_g, patch_size_g)
    best_patch_gray = image[best_y_g:best_y_g + patch_size_g, best_x_g:best_x_g + patch_size_g]


    # color로 판별
    height, width, _ = image.shape

    patch_size = min(height -image_size_adj, width - image_size_adj)  # 1:1 비율의 패치 크기 결정(원본보다 20 작게)
    target_hist = calculate_histogram(image)  # 원본 이미지의 히스토그램 계산

    best_x, best_y = find_best_match(image, target_hist, patch_size)
    best_patch_color = image[best_y:best_y + patch_size, best_x:best_x + patch_size]

    return best_patch_gray, best_patch_color


In [39]:
pic_num = 25

test_image_path = df.iloc[15]['file_path']
test_image = cv2.imread(test_image_path)

# 원본
print('원본')
cv2_imshow(test_image)
# gray 이미지로 선택하기
print('gray 이미지로 선택')
cv2_imshow(extract_best_patch(test_image)[0])
# 컬러 이미지로 선택하기
print('컬러 이미지로 선택')
cv2_imshow(extract_best_patch(test_image)[1])


원본


AttributeError: 'NoneType' object has no attribute 'clip'

In [40]:
test_image_path

'/content/drive/MyDrive/project3/data/traindata/crop_image/밥/유부초밥/Img_075_0057_crop.jpg'

In [41]:
# 최선의 그림을 찾아서 저장하기
def select_best_patch_image(image_path):

    # 디렉토리 생성
    color_image_path = image_path.replace('raw_image', 'selected_image_by_color_hist').replace('Img', 'select_color')
    os.makedirs(os.path.dirname(color_image_path), exist_ok=True)

    gray_image_path = image_path.replace('raw_image', 'selected_image_by_gray_hist').replace('Img', 'select_gray')
    os.makedirs(os.path.dirname(gray_image_path), exist_ok=True)


    img = cv2.imread(image_path)
    if img is not None:

        # extract_best_patch(img[0]) -> gray hist로 선택
        # gray_select 만들기
        best_patch_gray = extract_best_patch(img)[0]
        # 저장하기
        cv2.imwrite(gray_image_path, best_patch_gray)

        # extract_best_patch(img[1]) -> color hist로 선택
        # color_select 만들기
        best_patch_color = extract_best_patch(img)[1]
        cv2.imwrite(color_image_path, best_patch_color)






In [45]:
for file_path in tqdm(df['file_path']):
    select_best_patch_image(file_path)



100%|██████████| 988/988 [22:32<00:00,  1.37s/it]
